## Imports

In [20]:
from geopy.geocoders import GoogleV3
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd
import numpy as np
import urllib3
import re
from textwrap import shorten
import os

In [21]:
df = pd.read_csv('dtown_Miami_map_template_popup_pin - Sheet1.csv')

## Image URL collector

In [22]:
# Drop last n rows of a df. In case there are sums that need clipped
# df.drop(df.tail(n).index, inplace=True)

In [23]:
df['image_links'] = ''
links = []
for url in df['story_link']:
    try:
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        images = soup.find_all('img')
        counter = 0
        for image in images:
            counter += 1
            if 'jpg' in image['src']:
                links.append(image['src'])
                if counter == len(images):
                    links.append('no image found')
                break
    except:
        links.append('no_image')

df['image_links'] = links

In [24]:
len(links)

9

## Article link formatter

In [25]:
df["short_description"] = df["description"].apply(lambda s: shorten(s, width=20, placeholder=""))
df['remaining_desc'] = df.apply(lambda row : row['description'].replace(str(row['short_description']), ''), axis=1)
df['remaining_desc']

df["description_link"] = '<a href="' + df["story_link"] + '" target="_blank" rel="noopener noreferrer">' + df["short_description"] + "</a>" + df["remaining_desc"]
df['description_link']

0    <a href="https://therealdeal.com/miami/2022/08...
1    <a href="https://therealdeal.com/miami/2022/07...
2    <a href="https://therealdeal.com/miami/2022/07...
3    <a href="https://therealdeal.com/miami/2022/07...
4    <a href="https://therealdeal.com/miami/2022/06...
5    <a href="https://therealdeal.com/miami/2022/03...
6    <a href="https://therealdeal.com/miami/2022/02...
7    <a href="https://therealdeal.com/miami/2022/02...
8    <a href="https://therealdeal.com/miami/2022/01...
Name: description_link, dtype: object

## Google Maps API Geolocater Setup

In [26]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [27]:
df['geo_address'] = df['full_address'] + ' Miami, FL'
df['loc'] = df['geo_address'].apply(geolocator.geocode, timeout=10)
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

## Correction Section

In [28]:
df.columns

Index(['full_address', 'developers', 'land_purchase_price', 'loan_amount',
       'condos_or_apts', 'num_of_units', 'description', 'story_link',
       'other_links', 'image_links', 'short_description', 'remaining_desc',
       'description_link', 'geo_address', 'loc', 'point', 'lat', 'lon',
       'altitude'],
      dtype='object')

In [29]:
df

,full_address,developers,land_purchase_price,loan_amount,condos_or_apts,num_of_units,description,story_link,other_links,image_links,short_description,remaining_desc,description_link,geo_address,loc,point,lat,lon,altitude
0,222 Northeast First Avenue,Namdar,40500000.0,195000000.0,apartments,1354.0,Namdar Group scored a $195 million loan last m...,https://therealdeal.com/miami/2022/08/25/namda...,https://therealdeal.com/miami/2022/08/26/namda...,https://s12.therealdeal.com/trd/m/up/2022/08/m...,Namdar Group scored,a $195 million loan last month to buy the dev...,"<a href=""https://therealdeal.com/miami/2022/08...","222 Northeast First Avenue Miami, FL","(222 NE 1st Ave, Miami, FL 33132, USA, (25.776...","(25.7764704, -80.1921083, 0.0)",25.776470,-80.192108,0.0
1,400 SE 2nd Ave,"Hyatt, Gencom",NaN,NaN,apartments,1764.0,Chicago-based Hyatt and Miami-based Gencom pla...,https://therealdeal.com/miami/2022/07/29/hyatt...,NaN,https://s11.therealdeal.com/trd/m/up/2022/07/M...,Chicago-based Hyatt,and Miami-based Gencom plan to redevelop the ...,"<a href=""https://therealdeal.com/miami/2022/07...","400 SE 2nd Ave Miami, FL","(400 SE 2nd Ave, Miami, FL 33131, USA, (25.770...","(25.7709263, -80.1910809, 0.0)",25.770926,-80.191081,0.0
2,56 Southwest First Street,"Lions Group NYC, Fortis Design + Build",NaN,NaN,apartments,675.0,Long Island-based Lions Group NYC and Miami-ba...,https://therealdeal.com/miami/2022/07/19/lions...,NaN,https://s11.therealdeal.com/trd/m/up/2022/07/m...,Long Island-based,Lions Group NYC and Miami-based Fortis Design...,"<a href=""https://therealdeal.com/miami/2022/07...","56 Southwest First Street Miami, FL","(56 SW 1st St, Miami, FL 33130, USA, (25.77316...","(25.7731691, -80.19454449999999, 0.0)",25.773169,-80.194544,0.0
3,225 North Miami Avenue,"Related Group, ROVR Development and BH Group",NaN,76000000.0,condos,343.0,"Currently under construction, financed by a $7...",https://therealdeal.com/miami/2022/07/07/devel...,NaN,https://s13.therealdeal.com/trd/m/up/2022/07/D...,Currently under,"construction, financed by a $76 million loan ...","<a href=""https://therealdeal.com/miami/2022/07...","225 North Miami Avenue Miami, FL","(225 N Miami Ave, Miami, FL 33128, USA, (25.77...","(25.7766108, -80.1935231, 0.0)",25.776611,-80.193523,0.0
4,Northwest Eighth Street between North Miami an...,Lalezarian Properties,43000000.0,150000000.0,condos,565.0,New York-based Lalezarian plans a 53-story tow...,https://therealdeal.com/miami/2022/06/29/lende...,NaN,https://s12.therealdeal.com/trd/m/up/2022/06/M...,New York-based,Lalezarian plans a 53-story tower with 565 co...,"<a href=""https://therealdeal.com/miami/2022/06...",Northwest Eighth Street between North Miami an...,"(NE 1st St & N Miami Ave, Miami, FL 33132, USA...","(25.7751385, -80.1936288, 0.0)",25.775139,-80.193629,0.0
5,1016 Northeast Second Avenue,"Naftali Group, Cara Real Estate Management",40500000.0,NaN,NaN,NaN,Naftali Group and Cara Real Estate plan two re...,https://therealdeal.com/miami/2022/03/28/nafta...,NaN,https://s12.therealdeal.com/trd/m/up/2022/03/m...,Naftali Group and,"Cara Real Estate plan two residential towers,...","<a href=""https://therealdeal.com/miami/2022/03...","1016 Northeast Second Avenue Miami, FL","(1016 NE 2nd Ave, Miami, FL 33132, USA, (25.78...","(25.784095, -80.19111, 0.0)",25.784095,-80.191110,0.0
6,1025 North Miami Avenue,Property Markets Group and E11even Partners,NaN,149000000.0,condos,NaN,Property Markets Group and E11even Partners pl...,https://therealdeal.com/miami/2022/02/03/pmg-e...,NaN,https://s11.therealdeal.com/trd/m/up/2022/02/M...,Property Markets,Group and E11even Partners plan a two-buildin...,"<a href=""https://therealdeal.com/miami/2022/02...","1025 North Miami Avenue Miami, FL","(1025 N Miami Ave, Miami, FL 33136, USA, (25.7...","(25.784388, -80.1937892, 0.0)",25.784388,-80.193789,0.0
7,700 North Miami Avenue,Witkoff and Monroe Capital,94000000.0,NaN,unknown,2200.0,"In February, Witkoff and Monroe Capital propos...",ht

## Correction section

In [30]:
# df.at[0,'Net decline in footprint, in sf']=('Almost 200K sf')
# df.at[1,'Address(es) of space company vacated or plans to vacate, in sf']=('690 East Middlefield Road, Mountain View. Synopsys is subletting the entire building to Waymo.')

### Change value to percent value
### df['percent_change'] = pd.Series(['{0:.2f}%'.format(val) for val in df['percent_change']], index = df.index)

## HTML popup formatter

In [31]:
df.columns

Index(['full_address', 'developers', 'land_purchase_price', 'loan_amount',
       'condos_or_apts', 'num_of_units', 'description', 'story_link',
       'other_links', 'image_links', 'short_description', 'remaining_desc',
       'description_link', 'geo_address', 'loc', 'point', 'lat', 'lon',
       'altitude'],
      dtype='object')

In [32]:
def popup_html(row):
    i = row
    address = df['full_address'].iloc[i]
    developer = df['developers'].iloc[i]
    image = df['image_links'].iloc[i]
    description = df['description_link'].iloc[i]
    
    html = '''<!DOCTYPE html>
    <html>
    <img src={} width="256" height="156">'''.format(image) + '''<br>______________________________________<br>
    Address: <strong>{}'''.format(address) + '''</strong><br>
    Developer: <strong>{}'''.format(developer) + '''</strong><br>
    <strong>{}'''.format(description) + '''</strong><br>
    </html>
    '''
    return html


### HTML reservoir

In [33]:
#     Developer(s): <strong><em>{}'''.format(developer) + '''</strong></em><br><br>

## Map Maker

In [34]:
df.description_link[0]

'<a href="https://therealdeal.com/miami/2022/08/25/namdar-nabs-195m-loan-for-downtown-miami-apartment-towers/" target="_blank" rel="noopener noreferrer">Namdar Group scored</a> a $195 million loan last month to buy the development site at 222 Northeast First Avenue and build two apartment towers. The New York-based real estate investment and development firm plans Namdar Towers to include one 41-story tower with 640 units and a 43-story tower with 714 units. Slate Property Group\'s affiliate Scale Lending provided the financing.'

In [35]:
import folium
import branca

f = folium.Figure(width=750, height=750)
m = folium.Map(location=df[["lat", "lon"]].mean().to_list(),zoom_start=14)

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Downtown Miami Resi Projects')

for i in range(0,len(df)):
    html = popup_html(i)
    iframe = branca.element.IFrame(html=html)
    popup = folium.Popup(folium.Html(html, script=True))
    folium.Marker([df['lat'].iloc[i],df['lon'].iloc[i]],
                 popup=popup).add_to(m)

m.get_root().html.add_child(folium.Element(title_html))
m.fit_bounds(bounds=df[['lat','lon']].mean().to_list(),max_zoom=15)
# m.add_to(f)
m

In [39]:
m.save('index.html')

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [40]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [41]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/dtown_miami
